In [3]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

%matplotlib inline

PATH = '../data/preprocess/'
file_name = '2005-2022 viral uveitis_220624.xlsx'

In [4]:
# Read data
df = pd.read_excel(PATH + file_name)

# Total number
print(df.shape)


(10000, 350)


In [5]:
# 진단일자는 전처리를 위해 넣어준다.
init_columns = ['연구등록번호','Diagnosis','CMV PCR[EYE]','HSV type I PCR[EYE]','HSV type II PCR[EYE]','VZV PCR[EYE]','진단일자','Gender','진단시점나이','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

feature_column = ['Diagnosis','진단시점나이','Gender','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']
print(len(init_columns))
print(len(df.columns))

df_init = df[init_columns]

24
350


In [6]:
# sort by 연구등록번호
df_init = df_init.sort_values(by=['연구등록번호','진단일자'])

# save to xlsx
# df_init.to_excel(PATH + 'init_data.xlsx', index=False)

# Information

In [7]:
df_init.info()
# Count missing values
print(df_init.isnull().sum())
# Count Diagnosis 0,1,2 
print(df_init['Diagnosis'].value_counts(normalize=True) * 100)
print(df_init['Diagnosis'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 9391 to 873
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      10000 non-null  int64  
 1   Diagnosis                   10000 non-null  int64  
 2   CMV PCR[EYE]                295 non-null    object 
 3   HSV type I PCR[EYE]         22 non-null     float64
 4   HSV type II PCR[EYE]        191 non-null    float64
 5   VZV PCR[EYE]                18 non-null     float64
 6   진단일자                        10000 non-null  object 
 7   Gender                      10000 non-null  int64  
 8   진단시점나이                      10000 non-null  int64  
 9   CMV IgM[Serum]              866 non-null    float64
 10  CMV IgG[Serum]              839 non-null    float64
 11  HSV IgM[Serum]              893 non-null    float64
 12  HSV IgG[Serum]              0 non-null      float64
 13  VZV IgM[Serum]              77

In [37]:
# 각 column 별 특징 판단
df_cluster = df[feature_column]

print(df_cluster.describe())


          Diagnosis        진단시점나이        Gender  CMV IgM[Serum]  \
count  10000.000000  10000.000000  10000.000000      866.000000   
mean       0.221600     45.348400      0.519700        0.099351   
std        0.613784     16.291586      0.499637        0.368809   
min        0.000000      0.000000      0.000000        0.000000   
25%        0.000000     34.000000      0.000000        0.000000   
50%        0.000000     46.000000      1.000000        0.000000   
75%        0.000000     57.000000      1.000000        0.109250   
max        2.000000     92.000000      1.000000        6.640000   

       CMV IgG[Serum]  HSV IgM[Serum]  HSV IgG[Serum]  VZV IgG[Serum]  \
count      839.000000      893.000000             0.0      775.000000   
mean        41.009743        0.264057             NaN      837.857535   
std         32.879870        0.428801             NaN     1034.776210   
min          0.000000        0.000000             NaN        0.000000   
25%         17.850000        0.

In [38]:
df_cluster_portion = df_cluster.groupby('Diagnosis').count()
df_cluster_portion = df_cluster_portion / df_cluster_portion.sum() * 100
print(df_cluster_portion)

           진단시점나이  Gender  CMV IgM[Serum]  CMV IgG[Serum]  HSV IgM[Serum]  \
Diagnosis                                                                   
0           88.05   88.05       91.916859       91.299166       91.713326   
1            1.74    1.74        3.117783        3.337306        3.471445   
2           10.21   10.21        4.965358        5.363528        4.815230   

           HSV IgG[Serum]  VZV IgM[Serum]  VZV IgG[Serum]  \
Diagnosis                                                   
0                     NaN       92.258065       91.225806   
1                     NaN        4.129032        4.129032   
2                     NaN        3.612903        4.645161   

           WBC COUNT[Whole blood]  Lymphocyte(#)[Whole blood]  \
Diagnosis                                                       
0                       88.511224                   88.462810   
1                        1.588310                    1.520661   
2                        9.900466               

## 데이터 확인

* '연구등록번호'는 동일하나 'Diagnosis' 가 다른 케이스 확인

In [43]:
df_check = df_init.groupby('연구등록번호')

flag_list = []

# group마다 다 동일한 'Diagnosis'이 있는지 확인
for name, group in df_check:
    flag = False
    for i in range(len(group)-1):
        if group.iloc[i]['Diagnosis'] != group.iloc[i+1]['Diagnosis']:
            flag = True
            break
    
    if flag :
        flag_list.append(name)



In [44]:
print(len(flag_list))
print(flag_list)


54
[1340694, 1650597, 1651044, 1787320, 2019670, 2255608, 2263768, 2580939, 2708467, 2928404, 2999386, 3020526, 3071154, 3249107, 3322426, 3380351, 3403218, 3593450, 3617531, 3767153, 3810313, 3850619, 3888182, 3924853, 4013778, 4024186, 4074938, 4117045, 4127695, 4129131, 4153111, 4297907, 4320777, 4441436, 4448924, 4515988, 4532082, 4540113, 4567104, 4660852, 4776068, 4867817, 4876243, 4940693, 5939457, 8488929, 8769082, 8918959, 9470306, 9824777, 9899921, 10392399, 10404588, 10417875]


* 휴먼 어노테이트 후 
* 검사관련 feature 가 다 비어 있는 row 제거

In [45]:
# after 인간 annotation
df_snd = pd.read_excel(PATH + 'snd_data2.xlsx')
print(df_snd.info())

# Count Diagnosis 0,1,2 
print(df_snd['Diagnosis'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9803 entries, 0 to 9802
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      9803 non-null   int64  
 1   Diagnosis                   9803 non-null   int64  
 2   CMV PCR[EYE]                233 non-null    object 
 3   HSV type I PCR[EYE]         15 non-null     float64
 4   HSV type II PCR[EYE]        157 non-null    float64
 5   VZV PCR[EYE]                15 non-null     float64
 6   진단일자                        9803 non-null   object 
 7   Gender                      9803 non-null   int64  
 8   진단시점나이                      9803 non-null   int64  
 9   CMV IgM[Serum]              850 non-null    float64
 10  CMV IgG[Serum]              823 non-null    float64
 11  HSV IgM[Serum]              876 non-null    float64
 12  HSV IgG[Serum]              0 non-null      float64
 13  VZV IgM[Serum]              760 n

In [46]:
feature_column = ['CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']


In [47]:
df_snd.dropna(subset=feature_column, inplace=True, how="all")

print(df_snd.info())
# Count Diagnosis 0,1,2 
print(df_snd['Diagnosis'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9534 entries, 0 to 9802
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      9534 non-null   int64  
 1   Diagnosis                   9534 non-null   int64  
 2   CMV PCR[EYE]                102 non-null    object 
 3   HSV type I PCR[EYE]         5 non-null      float64
 4   HSV type II PCR[EYE]        81 non-null     float64
 5   VZV PCR[EYE]                7 non-null      float64
 6   진단일자                        9534 non-null   object 
 7   Gender                      9534 non-null   int64  
 8   진단시점나이                      9534 non-null   int64  
 9   CMV IgM[Serum]              850 non-null    float64
 10  CMV IgG[Serum]              823 non-null    float64
 11  HSV IgM[Serum]              876 non-null    float64
 12  HSV IgG[Serum]              0 non-null      float64
 13  VZV IgM[Serum]              760 n

In [48]:
# WBC COUNT[Whole blood] 제거
feature_column2 = ['CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

df_snd.dropna(subset=feature_column2, inplace=True, how="all")

print(df_snd.info())
# Count Diagnosis 0,1,2 
print(df_snd['Diagnosis'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9172 entries, 0 to 9802
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      9172 non-null   int64  
 1   Diagnosis                   9172 non-null   int64  
 2   CMV PCR[EYE]                101 non-null    object 
 3   HSV type I PCR[EYE]         5 non-null      float64
 4   HSV type II PCR[EYE]        81 non-null     float64
 5   VZV PCR[EYE]                7 non-null      float64
 6   진단일자                        9172 non-null   object 
 7   Gender                      9172 non-null   int64  
 8   진단시점나이                      9172 non-null   int64  
 9   CMV IgM[Serum]              850 non-null    float64
 10  CMV IgG[Serum]              823 non-null    float64
 11  HSV IgM[Serum]              876 non-null    float64
 12  HSV IgG[Serum]              0 non-null      float64
 13  VZV IgM[Serum]              760 n

In [49]:
df_snd.to_excel(PATH + 'third_data.xlsx', index=False)